#上場株式データ取得


参考サイト

https://datatechlog.com/how-to-retrieve-stock-price-and-other-info-using-yfinance/

https://algotrading101.com/learn/yfinance-guide/



■抽出手順

全3763社（code.csv）から3000時点以上の値を有する会社を抽出する

時間がかかるため、分割して実行する（それぞれ約15分かかる）

range(0,1000)    →    0 ~  999 → df_0.csv

range(1000,2000) → 1000 ~ 1999 → df_1.csv

range(2000,3000) → 2000 ~ 2999 → df_2.csv

range(3000,3763) → 3000 ~ 3762 → df_3.csv


■抽出結果

df_0 → コード：1301 ~ 3919 → 716社 * 2640時点

df_1 → コード：3941 ~ 6236 → 621社 * 2658時点

df_2 → コード：6245 ~ 7983 → 727社 * 2709時点

df_3 → コード：7984 ~ 9997 → 666社 * 2689時点

#ライブラリ読み込み

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [4]:
# ライブラリの読み込み
import yfinance as yf
import pandas as pd
import numpy as np

In [5]:
code = pd.read_csv('/content/drive/MyDrive/data/code/code.csv')

#範囲

In [6]:
range_start = 0
range_end = 1000

#関数定義

In [7]:
def df_range(range_start, range_end):

  # 3000時点以上の時系列を有する株価時系列を抽出（i = 0 スタート）
  code_dict = {}
  j = 0
  i = range_start

  while True:
    df = yf.download(str(code.iloc[i,0]) + '.T', start='2000-01-01', end='2021-12-31')['Adj Close']
    print(i, code.iloc[i,0], code.iloc[i,1], df.shape)
    next_start = i + 1

    if len(df) > 3000:
      code_dict[j] = [code.iloc[i,0], code.iloc[i,1]]
      j += 1
      break
    i += 1

  for i in range(next_start, range_end):
    df_app = yf.download(str(code.iloc[i,0]) + '.T', start='2000-01-01', end='2021-12-31')['Adj Close']
    print(i, code.iloc[i,0], code.iloc[i,1], df_app.shape)
    if len(df_app) > 3000:
      df = pd.merge(df, df_app, how = 'inner', on = 'Date')
      code_dict[j] = [code.iloc[i,0], code.iloc[i,1]]
      j += 1

  list = []

  for i in range(len(code_dict)):
    list.append(code_dict[i][0])
  df.index.name = 'timestamp'
  df = df.set_axis(list, axis = 'columns')

  return df

# 実行

In [8]:
df = df_range(range_start, int(range_end/10))

for i in range(1, 10):
  df = pd.merge(df, df_range(range_start + i * 100, range_start + (i+1)*100), how = 'inner', on = 'timestamp')

[*********************100%***********************]  1 of 1 completed
0 1301 極洋 (5253,)
[*********************100%***********************]  1 of 1 completed
1 1332 日本水産 (5407,)
[*********************100%***********************]  1 of 1 completed
2 1333 マルハニチロ (1917,)
[*********************100%***********************]  1 of 1 completed
3 1375 雪国まいたけ (315,)
[*********************100%***********************]  1 of 1 completed
4 1376 カネコ種苗 (3437,)
[*********************100%***********************]  1 of 1 completed
5 1377 サカタのタネ (5253,)
[*********************100%***********************]  1 of 1 completed
6 1379 ホクト (5250,)
[*********************100%***********************]  1 of 1 completed
7 1380 秋川牧園 (5010,)
[*********************100%***********************]  1 of 1 completed
8 1381 アクシーズ (4884,)
[*********************100%***********************]  1 of 1 completed
9 1382 ホーブ (4059,)
[*********************100%***********************]  1 of 1 completed
10 1383 ベルグアース (2491,)
[**************

In [9]:
df.to_csv("df_0.csv")

from google.colab import files
files.download('df_0.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>